## Quicktour

参数高效微调（PEFT Parameter-Efficient Fine-Tuning​​）为大型预训练模型提供了参数高效的微调方法。传统做法是针对每个下游任务对模型的所有参数进行微调，但由于如今模型的参数数量庞大，这种方法的成本越来越高，也越来越不切实际。相反，训练较少的提示参数或使用诸如低秩适应（LoRA low-rank adaptation）之类的重新参数化方法来减少可训练参数的数量会更高效。

本快速入门将向您展示 PEFT 的主要功能，以及如何在普通消费者设备上训练或运行通常无法触及的大型模型的推理。

### Train

每种参数高效微调（PEFT）方法都由一个 PeftConfig 类定义，该类存储构建PeftModel所需的所有重要参数。例如，若要使用低秩适应（LoRA）进行训练，请加载并创建一个 LoraConfig 类，并指定以下参数：
- task_type: 目标任务
- inference_mode：是否使用模型来进行预测
- r: 低秩矩阵的维度
- lora_alpha：低秩矩阵的缩放因子
- lora_dropout：LoRA 层的丢弃概率

In [1]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, 
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.8)

一旦设置好 LoraConfig，就可以使用 get_peft_model() 函数创建一个 PeftModel。该函数需要一个基础模型（可以从 Transformers 库中加载）以及包含用于使用 LoRA 训练模型的参数的 LoraConfig。

加载您想要微调的基础模型。

In [3]:
from transformers import AutoModelForSeq2SeqLM

checkpoint = "bigscience/mt0-large"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

model.safetensors:   0%|          | 10.5M/4.92G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/69/cd/69cd50da0d4ff75491317a19d706f489d842f63c0e2d070a45087b5df49c299b/4f9e1961a362116b17c9e33a458394baed48f9147de767952086d5c4c662c018?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1747125868&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzEyNTg2OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82OS9jZC82OWNkNTBkYTBkNGZmNzU0OTEzMTdhMTlkNzA2ZjQ4OWQ4NDJmNjNjMGUyZDA3MGE0NTA4N2I1ZGY0OWMyOTliLzRmOWUxOTYxYTM2MjExNmIxN2M5ZTMzYTQ1ODM5NGJhZWQ0OGY5MTQ3ZGU3Njc5NTIwODZkNWM0YzY2MmMwMTg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=oNTRVRV7cb5S6OqcPnjFQT0FFjxbNA4koew4wv665j8KnndpK4DJXcHtZ0blilUYUf%7EIaxQ6dxLdKqBTYlHK3i8yHQsDP8m22UYHFtgKpCPNyolbRSrhzpZH8tQ25r166EbhYti2iy4VkqpnaN--DIo%7E-sCKPPkWcme93mL-vYMyFSlSGKlR0613UWBwx5Cnjv8gaWCMUNir2GF6Oyn8eBEU1vQsQ24IaL7cVm74doOuzt3C7Jj9f2lenNzCf42Wklv9cl6

model.safetensors:   0%|          | 21.0M/4.92G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/69/cd/69cd50da0d4ff75491317a19d706f489d842f63c0e2d070a45087b5df49c299b/4f9e1961a362116b17c9e33a458394baed48f9147de767952086d5c4c662c018?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1747125868&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzEyNTg2OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82OS9jZC82OWNkNTBkYTBkNGZmNzU0OTEzMTdhMTlkNzA2ZjQ4OWQ4NDJmNjNjMGUyZDA3MGE0NTA4N2I1ZGY0OWMyOTliLzRmOWUxOTYxYTM2MjExNmIxN2M5ZTMzYTQ1ODM5NGJhZWQ0OGY5MTQ3ZGU3Njc5NTIwODZkNWM0YzY2MmMwMTg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=oNTRVRV7cb5S6OqcPnjFQT0FFjxbNA4koew4wv665j8KnndpK4DJXcHtZ0blilUYUf%7EIaxQ6dxLdKqBTYlHK3i8yHQsDP8m22UYHFtgKpCPNyolbRSrhzpZH8tQ25r166EbhYti2iy4VkqpnaN--DIo%7E-sCKPPkWcme93mL-vYMyFSlSGKlR0613UWBwx5Cnjv8gaWCMUNir2GF6Oyn8eBEU1vQsQ24IaL7cVm74doOuzt3C7Jj9f2lenNzCf42Wklv9cl6

OSError: bigscience/mt0-large does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

使用 `get_peft_model()` 函数将基础模型和 `peft_config` 包装起来以创建一个 `PeftModel`。要了解模型中可训练参数的数量，请使用 `print_trainable_parameters` 方法。

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)